In [23]:
def unique_oligos_beta(full, select, bp = 30):
    '''Finds unique oligos of a particular length
    within a particular region of a repetitive elemement'''
    full_list = []
    for i in range(len(full) - bp):
        full_list.append(full[i:i+bp])
    full_set = [y for y in full_list if full_list.count(y) == 1]
    uniques = []
    for i in full_set:
        if i in select:
            uniques.append(i)
    if len(uniques) == 0:
        print('I\'m sorry, there are no unique oligos of length ' 
             + str(bp) + ' in the selection provided')
    else:
        print('The following sequence(s) of length ' + str(bp) + 
             ' are unique to your sequence')
        print(uniques)

In [35]:
unique_oligos_beta('''GCCACCATGGAATGGAATCATCGCACTGAAATCTTCCCAGGAACATCTGCAAGAAT
              AAATCCTAAACCAGGAGATCCCTGTTCAGATCAGCTTCAGGAACAACATGTTGATTC
              ACAGAACAAAAATGACAAGGCCAGCAATGAAGTAAAAAGAAAATCCAAACCCAGGCA
              GAGGATTTCAACAACCTTTCCCAGCACACTCAAAGAACAAATGAGATCTGAGGAAAG
              TAAGAGAACTGTGGAAGAGCTCAGAACAGGCCAGACAACAAATACAGAGGACACAGT
              CAAATCATTTATTGCATCAGAAATCTCAAGTATTGAAAGACAATGTGGGCAATATTT
              CAGTGATAAGTCAAATGTCAATGAGCACCAGAAGACACACACAGGGGAGAAGCCCTA
              TGTTTGCAGGGAGTGTGGGCGGGGCTTTACACAGAACTCACACCTCATCCAGCACCA
              GAGGACACACACAGGGGAGAAGCCCTATGTTTGCAGGGAGTGTGGGCGGGGCTTTAC
              ACAGAAGTCAGACCTCATCAAGCACCAGAGGACACACACAGGGGAGAAGCCCTATGT
              TTGCAGGGAGTGTGGGCGGGGCTTTACACAGAAGTCAGACCTCATCAAGCACCAGAG
              GACACACACAGGGGAGAAGCCCTATGTTTGCAGGGAGTGTGGGCGGGGCTTTACACA
              GAAGTCAGTCCTCATCAAGCACCAGAGGACACACACAGGGGAGAAGCCCTATGTTTG
              CAGGGAGTGTGGGCGGGGCTTTACACAGAAGTCAGTCCTCATCAAGCACCAGAGGACA
              CACACAGGGGAGAAGCCCTATGTTTGCAGGGAGTGTGGGCGGGGCTTTACAGCGAAGT
              CAGTCCTCATCCAGCACCAGAGGACACACACAGGGGAGAAGCCCTATGTTTGCAGGGA
              GTGTGGGCGGGGCTTTACAGCGAAGTCAAACCTCATCCAGCACCAGAGGACACACACA
              GGGGAGAAGCCCTATGTTTGCAGGGAGTGTGGGCGGGGCTTTACAGCGAAGTCAGTCC
              TCATCCAGCACCAGAGGACACACACAGGGGAGAAGCCCTATGTTTGCAGGGAGTGTGG
              GCGGGGCTTTACAGCGAAGTCAGTCCTCATCCAGCACCAGAGGACACACACAGGGGAGA
              AGCCCTATGTTTGCAGGGAGTGTGGGCGGGGCTTTACACAGAAGTCAAACCTCATCAAG
              CACCAGAGGACACACACAGGGGAGAAGCCCTATGTTTGCAGGGAGTGTGGGTGGGGCT
              TTACACAGAAGTCAG''', '''GTTTGCAGGGAGTGTGGGCGGGGCTTTACACAGAA
              GTCAGACCTCATCAAGCACCAGAGGACACAC''')

The following sequence(s) of length 30 are unique to your sequence
['GTCAGACCTCATCAAGCACCAGAGGACACA', 'TCAGACCTCATCAAGCACCAGAGGACACAC']


In [57]:
def unique_oligos(full, select, lowbp = 15, highbp = 30, lowtm = 55, hightm = 65):
    '''Finds unique oligos of a particular length
    within a particular region of a repetitive elemement'''
    full_list = []
    for n in range(lowbp, highbp):
        for i in range(len(full) - n):
            full_list.append(full[i:i+n])
    full_set = [y for y in full_list if full_list.count(y) == 1]
    if len(full_set) < 1:
        raise ValueError('Unfortunately, there are no unique oligos between' + str(lowbp)
             + ' and ' + str(highbp) + ' bp in the full sequence')
    from Bio.SeqUtils import MeltingTemp as mt
    from Bio.Seq import Seq
    temp_adjust = [y for y in full_set if lowbp < mt.Tm_NN(Seq(y)) < highbp]
    if len(temp_adjust) < 1:
        raise ValueError('Unfortunately there are no unique oligos with Tm values between '
              + str(lowtm) + ' and ' + str(hightm) + 'degrees Celsius in the sequence')
    uniques = []
    for i in temp_adjust:
        if i in select:
            uniques.append(i)
    if len(uniques) < 1:
        print('''I\'m sorry, there are no oligos that fit the provided parameters''')
    else:
        print('''The following sequence(s) are unique to your sequence and fit the
          parameters provided''')
        print(uniques)

In [58]:
unique_oligos('''GCCACCATGGAATGGAATCATCGCACTGAAATCTTCCCAGGAACATCTGCAAGAAT
              AAATCCTAAACCAGGAGATCCCTGTTCAGATCAGCTTCAGGAACAACATGTTGATTC
              ACAGAACAAAAATGACAAGGCCAGCAATGAAGTAAAAAGAAAATCCAAACCCAGGCA
              GAGGATTTCAACAACCTTTCCCAGCACACTCAAAGAACAAATGAGATCTGAGGAAAG
              TAAGAGAACTGTGGAAGAGCTCAGAACAGGCCAGACAACAAATACAGAGGACACAGT
              CAAATCATTTATTGCATCAGAAATCTCAAGTATTGAAAGACAATGTGGGCAATATTT
              CAGTGATAAGTCAAATGTCAATGAGCACCAGAAGACACACACAGGGGAGAAGCCCTA
              TGTTTGCAGGGAGTGTGGGCGGGGCTTTACACAGAACTCACACCTCATCCAGCACCA
              GAGGACACACACAGGGGAGAAGCCCTATGTTTGCAGGGAGTGTGGGCGGGGCTTTAC
              ACAGAAGTCAGACCTCATCAAGCACCAGAGGACACACACAGGGGAGAAGCCCTATGT
              TTGCAGGGAGTGTGGGCGGGGCTTTACACAGAAGTCAGACCTCATCAAGCACCAGAG
              GACACACACAGGGGAGAAGCCCTATGTTTGCAGGGAGTGTGGGCGGGGCTTTACACA
              GAAGTCAGTCCTCATCAAGCACCAGAGGACACACACAGGGGAGAAGCCCTATGTTTG
              CAGGGAGTGTGGGCGGGGCTTTACACAGAAGTCAGTCCTCATCAAGCACCAGAGGACA
              CACACAGGGGAGAAGCCCTATGTTTGCAGGGAGTGTGGGCGGGGCTTTACAGCGAAGT
              CAGTCCTCATCCAGCACCAGAGGACACACACAGGGGAGAAGCCCTATGTTTGCAGGGA
              GTGTGGGCGGGGCTTTACAGCGAAGTCAAACCTCATCCAGCACCAGAGGACACACACA
              GGGGAGAAGCCCTATGTTTGCAGGGAGTGTGGGCGGGGCTTTACAGCGAAGTCAGTCC
              TCATCCAGCACCAGAGGACACACACAGGGGAGAAGCCCTATGTTTGCAGGGAGTGTGG
              GCGGGGCTTTACAGCGAAGTCAGTCCTCATCCAGCACCAGAGGACACACACAGGGGAGA
              AGCCCTATGTTTGCAGGGAGTGTGGGCGGGGCTTTACACAGAAGTCAAACCTCATCAAG
              CACCAGAGGACACACACAGGGGAGAAGCCCTATGTTTGCAGGGAGTGTGGGTGGGGCT
              TTACACAGAAGTCAG''', '''GTTTGCAGGGAGTGTGGGCGGGGCTTTACACAGAA
              GTCAGACCTCATCAAGCACCAGAGGACACAC''')

I'm sorry, there are no oligos that fit the provided parameters
